In [1]:
import os
import random
import numpy as np
import pandas as pd
import multiprocessing
import tensorflow as tf
import keras
from scipy import signal
from itertools import starmap
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from model_src.Resnet import ResNet34
from model_src.Unet import Unet
from model_src.DilatedConv import RespDNN
from model_src.BianResnet import ResNet
import matplotlib.pyplot as plt
print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
DATA_PATH = '../../DataLake/stMary'
kf = KFold(n_splits=5)

2023-06-02 18:00:01.107365: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-02 18:00:01.154068: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 18:00:01.802034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
def generate_dataset(arg_pleths, arg_resps, fs=125, shift_factor=4):
    import copy
    dataset = []
    window_size = fs * 60 # 7500
    shift = int(window_size/shift_factor)
    samples_len = len(arg_pleths)

    cpy_resps = copy.deepcopy(arg_resps)
    cpy_pleths = copy.deepcopy(arg_pleths)

    for j in range(samples_len):
        rr = cpy_resps[j]; ppg = cpy_pleths[j]

        rr['offset'] = (rr['offset']-rr['offset'].min())/1000
        size_lim = int(fs * np.ceil(rr['offset'].max()))
        ppg = ppg[:size_lim]
        shift_n_times = int((len(ppg)-window_size)/shift)+1

        samp_rr = [len(rr.loc[ (rr['offset']>=0+(int(shift/fs)*i)) & ((rr['offset']<int(window_size/fs)+(int(shift/fs)*i))) ]) for i in range(shift_n_times)]
        samp_ppg = [ppg[0+(shift*i):window_size+(shift*i)] for i in range(shift_n_times)]

        for i in range(len(samp_ppg)):
            temp = []
            temp.append(samp_ppg[i])
            temp.append(samp_rr[i])
            dataset.append(temp)

    return dataset


def interpolation(x, input):
    x0 = int(np.floor(x))
    y0 = input[x0]
    x1 = int(np.ceil(x))
    y1 = input[x1]
    y = (y1-y0)*(x-x0) + y0
    return y


def signal_resample(input_signal, org_fs, new_fs, method='interpolation'):
    output_signal = []
    new_x = np.arange(0, len(input_signal), org_fs/new_fs)
    
    if method == 'interpolation': 
        interp = interpolation

    for x in new_x:
        y = interp(x, input_signal)
        output_signal.append(y)

    return np.asarray(output_signal)


def preprocessing(targets=None):
    print('Extract PLETH/RESP')
    pleths = [pd.read_csv(f'{DATA_PATH}/{sid}/pleth.csv', header=None, names=['sid', 'offset', 'pleth']).pleth.values for sid in targets.id.unique()]
    resps = [pd.read_csv(f'{DATA_PATH}/{sid}/respirationTimeline.csv', header=None, names=['sid', 'offset']) for sid in targets.id.unique()]

    # Before filtering: Check NaN
    for pleth in pleths:
        if any(np.isnan(pleth)):
            print('check')

    # Before filtering: Convert type as np.int16
    pleths = list(map(lambda pleth: pleth.astype(np.float32), pleths))


    print('Init Preprocessing: Filtering')
    taps = signal.firwin(numtaps=400, cutoff=[0.5, 8.0], window='hamming', pass_zero=False, fs=125)
    w, h = signal.freqz(taps)
    pool = multiprocessing.Pool(processes=40)
    filtered_pleths = pool.starmap(signal.filtfilt, [(taps, 1.0, pleth) for pleth in pleths])
    pool.close()
    pool.join()


    print('Init Preprocessing: Windowing')
    dataset = generate_dataset(filtered_pleths, resps, shift_factor=60)


    print('Init Preprocessing: Resampling')
    pool = multiprocessing.Pool(processes=40)
    result = pool.starmap(signal_resample, [(pleth[0], 125, 30) for pleth in dataset])
    pool.close()
    pool.join()

    res_pleth = []
    res_resp = []
    for i in range(len(dataset)):
        res_pleth.append(result[i])
        res_resp.append(dataset[i][1])

    return np.array(res_pleth), np.array(res_resp)


def prepare_modeling(dataset=None, batchsize=None):
    print(f'Prepare modeling')
    pleths = []
    resps = []
    for ppg, rr in dataset:
        pleths.append(ppg.astype(np.float32))
        resps.append(rr)
    pleths = np.asarray(pleths)
    resps = np.asarray(resps)
    print(pleths.shape, resps.shape)

    scaler = MinMaxScaler()
    scaled_pleths = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in pleths])
    print(scaled_pleths.shape, type(scaled_pleths[0][0][0]))

    x, y = scaled_pleths[:], resps[:]

    return tf.data.Dataset.from_tensor_slices((x, y)).batch(batchsize)

In [3]:
subjects = pd.read_csv(f'{DATA_PATH}/patients.csv')
patients = subjects.loc[subjects['diagnosis']!='0']
train_patients = patients.sample(frac=1, random_state=42)

In [21]:
EPOCHS = 1000
BATCH_SIZE = 256
LR = 0.001
callbacks = [
    EarlyStopping(monitor='val_loss', patience=32),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

In [5]:
counter = 1
raw_dataset = {
    'train': [],
    'val': []
}
dataset = {
    'train': [],
    'val': []
}
for train_idx, val_idx in kf.split(train_patients):
    print(f'\n{counter}th K-fold')
    counter = counter + 1
    # 이렇게 하는 이유는 Train과 Validation을 완벽히 구별시키기 위함이다.
    X_train, y_train = preprocessing(train_patients.iloc[train_idx])
    print(X_train.shape, y_train.shape)
    X_train_diff1 = np.expand_dims(np.array([np.convolve(X_train[i].reshape(-1), [-1, 0, 1], mode='same') for i in range(X_train.shape[0])]), axis=2)
    X_train_diff2 = np.expand_dims(np.array([np.convolve(X_train_diff1[i].reshape(-1), [-1, 0, 1], mode='same') for i in range(X_train_diff1.shape[0])]), axis=2)
    print(X_train.shape, X_train_diff1.shape, X_train_diff2.shape, y_train.shape)

    scaler = MinMaxScaler()
    scaled_X_train = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in X_train], dtype=np.float32)
    scaled_X_train_diff1 = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in X_train_diff1], dtype=np.float32)
    scaled_X_train_diff2 = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in X_train_diff2], dtype=np.float32)
    print(scaled_X_train.shape, type(scaled_X_train[0][0][0]))
    print(scaled_X_train_diff1.shape, type(scaled_X_train_diff1[0][0][0]))
    print(scaled_X_train_diff2.shape, type(scaled_X_train_diff2[0][0][0]))

    scaled_X_train = np.concatenate([scaled_X_train, scaled_X_train_diff1, scaled_X_train_diff2], axis=2)
    print(scaled_X_train.shape)

    train_dataset = tf.data.Dataset.from_tensor_slices((scaled_X_train, y_train)).batch(BATCH_SIZE)    



    X_val, y_val = preprocessing(train_patients.iloc[val_idx])
    print(X_val.shape, y_val.shape)
    X_val_diff1 = np.expand_dims(np.array([np.convolve(X_val[i].reshape(-1), [-1, 0, 1], mode='same') for i in range(X_val.shape[0])]), axis=2)
    X_val_diff2 = np.expand_dims(np.array([np.convolve(X_val_diff1[i].reshape(-1), [-1, 0, 1], mode='same') for i in range(X_val_diff1.shape[0])]), axis=2)
    print(X_val.shape, X_val_diff1.shape, X_val_diff2.shape, y_val.shape)

    scaler = MinMaxScaler()
    scaled_X_val = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in X_val], dtype=np.float32)
    scaled_X_val_diff1 = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in X_val_diff1], dtype=np.float32)
    scaled_X_val_diff2 = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in X_val_diff2], dtype=np.float32)
    print(scaled_X_val.shape, type(scaled_X_val[0][0][0]))
    print(scaled_X_val_diff1.shape, type(scaled_X_val_diff1[0][0][0]))
    print(scaled_X_val_diff2.shape, type(scaled_X_val_diff2[0][0][0]))

    scaled_X_val = np.concatenate([scaled_X_val, scaled_X_val_diff1, scaled_X_val_diff2], axis=2)
    print(scaled_X_val.shape)

    val_dataset = tf.data.Dataset.from_tensor_slices((scaled_X_val, y_val)).batch(BATCH_SIZE)


    raw_dataset['train'].append(X_train)
    raw_dataset['val'].append(X_val)

    # train_dataset = prepare_modeling(X_train, batchsize=256)
    # val_dataset = prepare_modeling(X_val, batchsize=256)

    dataset['train'].append(train_dataset)
    dataset['val'].append(val_dataset)


1th K-fold
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
(5180, 1800) (5180,)
(5180, 1800) (5180, 1800, 1) (5180, 1800, 1) (5180,)
(5180, 1800, 1) <class 'numpy.float32'>
(5180, 1800, 1) <class 'numpy.float32'>
(5180, 1800, 1) <class 'numpy.float32'>
(5180, 1800, 3)
Extract PLETH/RESP
Init Preprocessing: Filtering


2023-06-02 18:01:27.044119: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14353 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5


Init Preprocessing: Windowing
Init Preprocessing: Resampling
(1328, 1800) (1328,)
(1328, 1800) (1328, 1800, 1) (1328, 1800, 1) (1328,)
(1328, 1800, 1) <class 'numpy.float32'>
(1328, 1800, 1) <class 'numpy.float32'>
(1328, 1800, 1) <class 'numpy.float32'>
(1328, 1800, 3)

2th K-fold
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
(5214, 1800) (5214,)
(5214, 1800) (5214, 1800, 1) (5214, 1800, 1) (5214,)
(5214, 1800, 1) <class 'numpy.float32'>
(5214, 1800, 1) <class 'numpy.float32'>
(5214, 1800, 1) <class 'numpy.float32'>
(5214, 1800, 3)
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
(1294, 1800) (1294,)
(1294, 1800) (1294, 1800, 1) (1294, 1800, 1) (1294,)
(1294, 1800, 1) <class 'numpy.float32'>
(1294, 1800, 1) <class 'numpy.float32'>
(1294, 1800, 1) <class 'numpy.float32'>
(1294, 1800, 3)

3th K-fold
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessi

In [28]:
import os
import keras
import tensorflow as tf
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Dense, BatchNormalization, Activation, Add, Flatten, Dropout, Multiply
print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


class RespBlock(Model):
    def __init__(self, filters, *args, **kwargs):
        super(RespBlock, self).__init__(*args, **kwargs)
        self.conv1 = Conv1D(filters=filters, kernel_size=3, strides=1, dilation_rate=1, padding='same')
        self.bn1 = BatchNormalization()

        self.conv2 = Conv1D(filters=filters, kernel_size=3, strides=1, dilation_rate=2, padding='same')
        self.bn2 = BatchNormalization()

        self.conv3 = Conv1D(filters=filters, kernel_size=3, strides=1, dilation_rate=3, padding='same')
        self.bn3 = BatchNormalization()

        # self.conv4 = Conv1D(filters=filters, kernel_size=3, strides=1, dilation_rate=5, padding='same')
        # self.bn4 = BatchNormalization()

        self.conv1x1 = Conv1D(filters=filters*2, kernel_size=1, strides=1, padding='same')
        self.bn1x1 = BatchNormalization()


    def call(self, inputs, training=None, mask=None): # 900, 32
        x1 = self.conv1(inputs)
        x1 = self.bn1(x1)
        x1 = Activation('relu')(x1)

        x2 = self.conv2(inputs)
        x2 = self.bn2(x2)
        x2 = Activation('relu')(x2)
        
        x3 = self.conv3(inputs)
        x3 = self.bn3(x3)
        x3 = Activation('relu')(x3)
        
        # x4 = self.conv4(inputs)
        # x4 = self.bn4(x4)
        # x4 = Activation('relu')(x4)

        x = Add()([x1, x2, x3])                       # 900, 64
        # x = Multiply()([x1, x2, x3])

        # x = self.conv1x1(x)
        # x = self.bn1x1(x)

        if inputs.shape[-1] != 1:
            inputs = tf.reduce_mean(inputs, axis=-1, keepdims=True)
            # inputs = self.conv1x1(inputs)
            # inputs = self.bn1x1(inputs)
           
        x = Add()([x, inputs])
        return x


class RespDNN2(Model):
    def __init__(self, *args, **kwargs):
        super(RespDNN2, self).__init__(*args, **kwargs)
        self.respblk = [RespBlock(32*i) for i in np.arange(1, 4)]
        self.dwnsamp = [Conv1D(32*i, kernel_size=3, strides=2, padding='same') for i in np.arange(1, 4)]
        self.bn = [BatchNormalization() for _ in range(3)]
        self.avgpool = AveragePooling1D(strides=2, padding='valid')
        self.dense1 = Dense(1000, activation='relu')
        self.dense2 = Dense(100, activation='relu')
        self.dense3 = Dense(1)

    @tf.function
    def call(self, inputs, training=None, mask=None):
        x = inputs[:, :, 0:1]
        x_diff1 = inputs[:, :, 1:2]
        x_diff2 = inputs[:, :, 2:3]

        for i in range(3):
           x = self.respblk[i](x)
           x = self.dwnsamp[i](x)
           x = self.bn[i](x)
           x = Activation('relu')(x)

        
        for i in range(3):
           x_diff1 = self.respblk[i](x_diff1)
           x_diff1 = self.dwnsamp[i](x_diff1)
           x_diff1 = self.bn[i](x_diff1)
           x_diff1 = Activation('relu')(x_diff1)


        for i in range(3):
           x_diff2 = self.respblk[i](x_diff2)
           x_diff2 = self.dwnsamp[i](x_diff2)
           x_diff2 = self.bn[i](x_diff2)
           x_diff2 = Activation('relu')(x_diff2)


        x = Multiply()([x, x_diff1, x_diff2])
        
        x = self.avgpool(x)
        x = Flatten()(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [29]:
train_losses = []
val_losses = []
for i in range(5):
    print(f'{i+1}th K-fold')
    # model = ResNet34()
    # model = ResNet()
    # model = Unet()
    model = RespDNN2()

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
        loss=keras.losses.MeanAbsoluteError(),
        metrics=keras.metrics.MeanAbsoluteError()
    )  

    # callbacks.append(ModelCheckpoint(f'../models/230531-Resnet-L34-stmary-KF{i}', monitor='val_loss', save_best_only=True))
    
    history = model.fit(
        dataset['train'][i],
        epochs=EPOCHS,
        callbacks=callbacks,
        validation_data=dataset['val'][i]
    )

    train_losses.append(min(history.history['loss']))
    val_losses.append(min(history.history['val_loss']))

1th K-fold
Epoch 1/1000


2023-06-02 19:28:46.046059: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/resp_block_32/conv1d_151/Conv1D_2/BatchToSpaceND_grad/SpaceToBatchND/resp_block_32/conv1d_151/Conv1D_2/BatchToSpaceND/block_shape' with dtype int32 and shape [1]
	 [[{{node gradients/resp_block_32/conv1d_151/Conv1D_2/BatchToSpaceND_grad/SpaceToBatchND/resp_block_32/conv1d_151/Conv1D_2/BatchToSpaceND/block_shape}}]]
2023-06-02 19:28:46.046158: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/resp_block_32/conv1d_151/Conv1D_2/BatchToSpaceND_grad/SpaceToBatchND/resp_block_32/conv1d_151/Conv1D_2/BatchToSpaceND/crops' with dtype i

21/21 [==============================] - 21s 400ms/step - loss: 10.2358 - mean_absolute_error: 10.2358 - val_loss: 16.1776 - val_mean_absolute_error: 16.1776 - lr: 0.0010
Epoch 2/1000
21/21 [==============================] - 7s 352ms/step - loss: 8.9802 - mean_absolute_error: 8.9802 - val_loss: 5.3154 - val_mean_absolute_error: 5.3154 - lr: 0.0010
Epoch 3/1000
21/21 [==============================] - 7s 355ms/step - loss: 6.2242 - mean_absolute_error: 6.2242 - val_loss: 9.6626 - val_mean_absolute_error: 9.6626 - lr: 0.0010
Epoch 4/1000
21/21 [==============================] - 7s 356ms/step - loss: 5.5958 - mean_absolute_error: 5.5958 - val_loss: 8.5399 - val_mean_absolute_error: 8.5399 - lr: 0.0010
Epoch 5/1000
16/21 [=====================>........] - ETA: 1s - loss: 3.9835 - mean_absolute_error: 3.9835

KeyboardInterrupt: 